In [2]:
import mysql.connector as mysql

db = mysql.connect(user='superheroadmin',
                   password='Passw0rd.',
                   database='superheroes')

In [6]:
%env MYSQL_USER=superheroadmin
%env MYSQL_PASSWORD=Passw0rd.
%env MYSQL_HOST_IP=localhost
%env MYSQL_PORT=3306

env: MYSQL_USER=superheroadmin
env: MYSQL_PASSWORD=Passw0rd.
env: MYSQL_HOST_IP=localhost
env: MYSQL_PORT=3306


In [7]:
import os
import mysql.connector as mysql

db_name = 'superheroes'

db = mysql.connect(user=os.environ['MYSQL_USER'],
                   password=os.environ['MYSQL_PASSWORD'],
                   host=os.environ['MYSQL_HOST_IP'],
                   port=os.environ['MYSQL_PORT'],
                   database=db_name)

In [27]:
cursor = db.cursor()

In [9]:
query = 'SELECT * FROM heroes;'
cursor.execute(query)

In [10]:
for r in cursor:
    print(r)

(1, 'Wonder Woman', 'Diana Prince', 'Themyscira', 'DC Comics', 183.13, 74.74, 'F', 1941, 'Blue', 'Black', 100.0, 'high')
(3, 'Wolverine', 'Logan', 'Alberta, Canada', 'Marvel Comics', 160.7, 135.21, 'M', None, 'Blue', 'Black', 35.0, 'good')
(4, 'Spider-Man', 'Peter Parker', 'New York, New York', 'Marvel Comics', 178.28, 74.25, 'M', None, 'Hazel', 'Brown', 55.0, 'high')
(5, 'Professor X', 'Charles Francis Xavier', 'New York, New York', 'Marvel Comics', 183.74, 86.89, 'M', 1963, 'Blue', 'No Hair', 10.0, 'high')


In [11]:
cursor.execute('DELETE FROM heroes;')
db.commit()

In [12]:
add_hero_query = ('INSERT INTO heroes '
                  '(id, name, identity, birth_place, publisher, height, weight, gender,'
                  ' first_appearance, eye_color, hair_color, strength, intelligence) '
                  'VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);')

In [13]:
wonder_woman = (1, 'Wonder Woman', 'Diana Prince', 'Themyscira',
                'DC Comics', 183.13, 74.74, 'F', 1941, 'Blue',
                'Black', 100, 'high')
cursor.execute(add_hero_query, wonder_woman)

In [14]:
db.commit()

In [15]:
heroes = [
    (3, 'Wolverine', 'Logan', 'Alberta, Canada',
     'Marvel Comics', 160.70, 135.21, 'M', None,
     'Blue', 'Black', 35, 'good'),
    (4, 'Spider-Man', 'Peter Parker', 'New York, New York',
     'Marvel Comics', 178.28, 74.25, 'M', None,
     'Hazel', 'Brown', 55, 'high'),
    (5, 'Professor X', 'Charles Francis Xavier', 'New York, New York',
     'Marvel Comics', 183.74, 86.89, 'M', 1963,
     'Blue', 'No Hair', 10, 'high')
]

cursor.executemany(add_hero_query, heroes)

In [16]:
db.commit()

In [17]:
query = 'SELECT name, identity, first_appearance FROM heroes;'
cursor.execute(query)
for name, identity, first_appearance in cursor:
    print('{} aka {} ({})'.format(name, identity, first_appearance))

Wonder Woman aka Diana Prince (1941)
Wolverine aka Logan (None)
Spider-Man aka Peter Parker (None)
Professor X aka Charles Francis Xavier (1963)


In [18]:
parametrized_query = 'SELECT name, identity FROM heroes WHERE eye_color = %s;'
cursor.execute(parametrized_query, ('Blue',))
for name, identity in cursor:
    print('{} aka {}'.format(name, identity))

Wonder Woman aka Diana Prince
Wolverine aka Logan
Professor X aka Charles Francis Xavier


In [19]:
#cursor.reset
#
#cursor.fetchall

In [20]:
#cursor.close()
#db.close()

True

In [22]:
import csv

with open('data/heroes.csv', 'r') as heroes_file:
    heroes_reader = csv.reader(heroes_file, delimiter=';', quotechar='\"')
    heroes = list(heroes_reader)[1:]

In [24]:
heroes[0]

['A-Bomb',
 'Richard Milhouse Jones',
 'Scarsdale, Arizona',
 'Marvel Comics',
 '203.21000000000001',
 '441.94999999999999',
 'M',
 '2008',
 'Yellow',
 'No Hair',
 '100',
 'moderate']

In [25]:
def exists_in_db(hero, cursor):
    cursor.execute('SELECT name FROM heroes WHERE name=%s', (hero,))
    cursor.fetchall()
    return cursor.rowcount > 0

In [28]:
exists_in_db('Wonder Woman', cursor)

True

In [29]:
exists_in_db('Wonder Wman', cursor)

False

In [31]:
def next_id(cursor):
    cursor.execute('SELECT max(id) FROM heroes;')
    return cursor.fetchone()[0] + 1

In [32]:
next_id(cursor)

6

In [34]:
def to_float(s):
    return float(s) if s else None

def to_int(s):
    return int(s) if s else None

def to_str(s, max_size=1000):
    return s.encode('ascii', 'ignore')[:max_size] if s else None

converters = 4 * [to_str] + \
             2 * [to_float] + \
             [to_str, to_int] + \
             2 * [lambda s: to_str(s, max_size=10)] + \
             [to_int, to_str]

def to_record(h):
    return [c(s) for c, s in zip(converters, h)]

In [35]:
for h in heroes:
    print('checking {}...'.format((h[0])), end=' ')
    if exists_in_db(h[0], cursor):
        print('already existing: skipping.')
    else:
        new_id = next_id(cursor)
        print('adding with new id {}'.format(new_id))
        cursor.execute(add_hero_query, [new_id]+to_record(h))

db.commit()

checking A-Bomb... adding with new id 6
checking Abraxas... adding with new id 7
checking Abomination... adding with new id 8
checking Adam Monroe... adding with new id 9
checking Agent 13... adding with new id 10
checking Air-Walker... adding with new id 11
checking Agent Bob... adding with new id 12
checking Abe Sapien... adding with new id 13
checking Abin Sur... adding with new id 14
checking Angela... adding with new id 15
checking Animal Man... adding with new id 16
checking Agent Zero... adding with new id 17
checking Colin Wagner... adding with new id 18
checking Angel Dust... adding with new id 19
checking Angel Salvadore... adding with new id 20
checking Zoom... adding with new id 21
checking Lady Deathstrike... adding with new id 22
checking Yoda... adding with new id 23
checking Zatanna... adding with new id 24
checking Yellowjacket II... adding with new id 25
checking Yellowjacket... adding with new id 26
checking Yellow Claw... adding with new id 27
checking Absorbing Man

checking Mach-IV... adding with new id 325
checking Luna... adding with new id 326
checking Man of Miracles... adding with new id 327
checking Man-Wolf... adding with new id 328
checking Magneto... adding with new id 329
checking Luke Skywalker... adding with new id 330
checking Longshot... adding with new id 331
checking Luke Campbell... adding with new id 332
checking Loki... adding with new id 333
checking Living Tribunal... adding with new id 334
checking Lobo... adding with new id 335
checking Luke Cage... adding with new id 336
checking Lightning Lad... adding with new id 337
checking Light Lass... adding with new id 338
checking Lex Luthor... adding with new id 339
checking Lightning Lord... adding with new id 340
checking Legion... adding with new id 341
checking Leech... adding with new id 342
checking Leader... adding with new id 343
checking Lady Bullseye... adding with new id 344
checking Leonardo... adding with new id 345
checking Kyle Rayner... adding with new id 346
chec

checking Batwoman V... adding with new id 664
checking Batman II... adding with new id 665
checking Batman... adding with new id 666
checking Battlestar... adding with new id 667
checking Beyonder... adding with new id 668
checking Batgirl V... adding with new id 669
checking Batgirl III... adding with new id 670
checking Batman... already existing: skipping.
checking Batgirl VI... adding with new id 671
checking Batgirl... adding with new id 672
checking Batgirl... already existing: skipping.
checking Bantam... adding with new id 673
checking Banshee... adding with new id 674
checking Black Panther... adding with new id 675
checking Azrael... adding with new id 676
checking Aztar... adding with new id 677
checking Beak... adding with new id 678
checking Aurora... adding with new id 679
checking Azazel... adding with new id 680
checking Atom II... adding with new id 681
checking Atom IV... adding with new id 682
checking Batgirl IV... adding with new id 683
checking Atom... adding with

In [36]:
import pandas as pd
df = pd.read_sql('SELECT * FROM heroes;', con=db, index_col='id')
df.head()

,name,identity,birth_place,publisher,height,weight,gender,first_appearance,eye_color,hair_color,strength,intelligence
id,,,,,,,,,,,,
1,Wonder Woman,Diana Prince,Themyscira,DC Comics,183.13,74.74,F,1941.0,Blue,Black,100.0,high
3,Wolverine,Logan,"Alberta, Canada",Marvel Comics,160.70,135.21,M,NaN,Blue,Black,35.0,good
4,Spider-Man,Peter Parker,"New York, New York",Marvel Comics,178.28,74.25,M,NaN,Hazel,Brown,55.0,high
5,Professor X,Charles Francis Xavier,"New York, New York",Marvel Comics,183.74,86.89,M,1963.0,Blue,No Hair,10.0,high
6,A-Bomb,Richard Milhouse Jones,"Scarsdale, Arizona",Marvel Comics,203.21,441.95,M,2008.0,Yellow,No Hair,100.0,moderate
